In [24]:
import pyodbc
import sqlite3
import pandas as pd


In [25]:
export_conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=(localdb)\\MSSQLLocalDB;'
    'DATABASE=projectsourcedatamodel;'
    'Trusted_Connection=yes;'
)

export_cursor = export_conn.cursor()

In [26]:
def clear_tables():
    tables = ["Purchasing_PurchaseOrderDetail", "Purchasing_PurchaseOrderHeader", "Purchasing_Vendor", "Sales_Order_Item", "Order_Details", "Bonus", "EmployeeTerritories", "[Order]", "Employee", "Production_BillOfMaterials", "Sales_SalesOrderDetail", "Production_Product", "Product", "Sales_SalesOrderHeader", "CustomerCustomerDemo", "Customer", "CustomerDemographics", "Sales_Customer", "Person_Address", "Sales_Store", "Person_Person", "Sales_SalesTerritory", "Territories", "State", "Region", "Category", "Suppliers", "Department", "Shippers"]

    for table in tables:
        export_cursor.execute(f"DELETE FROM {table}")
        export_conn.commit()


    print("Alle tabellen zijn geleegd.")

clear_tables()


Alle tabellen zijn geleegd.


In [27]:
nconn =  pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=(localdb)\\MSSQLLocalDB;'
    'DATABASE=Northwind;'
    'Trusted_Connection=yes;'
)

nconn_cursor = nconn.cursor()

aenconn = sqlite3.connect('c:/Users/ibrah/Documents/DEDS_Project/src/Data/Raw/aenc.sqlite')
advconn = sqlite3.connect('c:/Users/ibrah/Documents/DEDS_Project/src/Data/Raw/adventureworks.sqlite')


In [28]:
def move_shippers():
    df = pd.read_sql_query("SELECT * FROM Shippers", nconn)
    for index, row in df.iterrows():
        try:
            query = f"INSERT INTO Shippers VALUES ({row['ShipperID']}, '{row['CompanyName'].replace("'", "''")}', '{row['Phone']}')"
            export_cursor.execute(query)
        except pyodbc.Error as e:
            print(f"Fout in Shippers: {e}")
    export_conn.commit()
move_shippers()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\3807513202.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM Shippers", nconn)


In [29]:
def move_department():
    aen_df = pd.read_sql_query("SELECT dept_id, dept_name, dept_head_id FROM Department", aenconn)
    adv_df = pd.read_sql_query("SELECT DepartmentID as dept_id, Name as dept_name, GroupName FROM HumanResources_Department", advconn)
    
    combined_df = pd.concat([
        aen_df[['dept_id', 'dept_name', 'dept_head_id']],
        adv_df[['dept_id', 'dept_name', 'GroupName']].assign(dept_head_id=None) 
    ]).drop_duplicates('dept_id')  
    
    for index, row in combined_df.iterrows():
        try:
            query = f"""
            INSERT INTO Department (
                Dept_id,
                Dept_name,
                Dept_head_id,
                GroupName,
                ModifiedDate
            ) VALUES (
                {row['dept_id']},
                '{row['dept_name'].replace("'", "''") if pd.notna(row['dept_name']) else ''}',
                {row['dept_head_id'] if pd.notna(row['dept_head_id']) else 'NULL'},
                {f"'{row['GroupName'].replace("'", "''")}'" if pd.notna(row['GroupName']) else 'NULL'},
                {'NULL'}   
            )
            """
            export_cursor.execute(query)
        except pyodbc.Error as e:
            print(f"Fout in Department insert (dept_id: {row['dept_id']}): {e}")
    
    export_conn.commit()

move_department()

In [30]:
def move_suppliers():
    df = pd.read_sql_query("SELECT * FROM Suppliers", nconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Suppliers (
                SupplierID, CompanyName, ContactName, ContactTitle,
                Address, City, Region, PostalCode, Country,
                Phone, Fax, HomePage
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            export_cursor.execute(query,
                row['SupplierID'],
                row['CompanyName'],
                row['ContactName'] if pd.notna(row['ContactName']) else None,
                row['ContactTitle'] if pd.notna(row['ContactTitle']) else None,
                row['Address'] if pd.notna(row['Address']) else None,
                row['City'] if pd.notna(row['City']) else None,
                row['Region'] if pd.notna(row['Region']) else None,
                row['PostalCode'] if pd.notna(row['PostalCode']) else None,
                row['Country'] if pd.notna(row['Country']) else None,
                row['Phone'] if pd.notna(row['Phone']) else None,
                row['Fax'] if pd.notna(row['Fax']) else None,
                row['HomePage'] if pd.notna(row['HomePage']) else None
            )
        except pyodbc.Error as e:
            print(f"Fout in Suppliers (SupplierID: {row['SupplierID']}): {e}")
    export_conn.commit()

move_suppliers()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\1321044012.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM Suppliers", nconn)


In [31]:
def move_category():
    adv_df = pd.read_sql_query("""
        SELECT 
            ProductCategoryID as CategoryID, 
            Name as CategoryName,
            NULL as Description,
            NULL as Picture
        FROM Production_ProductCategory
    """, advconn)

    n_df = pd.read_sql_query("""
        SELECT 
            CategoryID,
            CategoryName,
            Description,
            Picture
        FROM Categories
    """, nconn)
    
    combined_df = pd.concat([adv_df, n_df], ignore_index=True)
    combined_df = combined_df.drop_duplicates(subset=['CategoryID'], keep='first')

    for index, row in combined_df.iterrows():
        try:
            description = None if pd.isna(row['Description']) else row['Description']
            
            query = """
            INSERT INTO Category (
                CategoryID,
                CategoryName,
                Description,
                Picture,
                rowguid,
                ModifiedDate
            ) VALUES (?, ?, ?, ?, NULL, NULL)
            """
            export_cursor.execute(query, 
                row['CategoryID'],
                row['CategoryName'],
                description,
                row['Picture'] if not pd.isna(row['Picture']) else None
            )
            
        except pyodbc.Error as e:
            print(f"Fout in Category insert (CategoryID: {row['CategoryID']}): {e}")
    
    export_conn.commit()
move_category()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\439097002.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  n_df = pd.read_sql_query("""


In [32]:
def move_region():
    df = pd.read_sql_query("SELECT * FROM Region", nconn)
    for index, row in df.iterrows():
        try:
            query = f"INSERT INTO Region VALUES ({row['RegionID']}, '{row['RegionDescription']}')"
            export_cursor.execute(query)
        except pyodbc.Error as e:
            print(f"Fout in Region: {e}")
    export_conn.commit()
move_region()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\794136371.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM Region", nconn)


In [33]:
def move_state():
    df = pd.read_sql_query("SELECT * FROM State", aenconn)
    for index, row in df.iterrows():
        try:
            query = f"""
            INSERT INTO State VALUES (
                '{row['state_id']}',
                '{row['state_name'].replace("'", "''")}',
                '{row['state_capital'].replace("'", "''")}',
                '{row['country'].replace("'", "''")}',
                '{row['region'].replace("'", "''")}'
            )
            """
            export_cursor.execute(query)
        except pyodbc.Error as e:
            print(f"Fout in State: {e}")
    export_conn.commit()
move_state()

In [34]:
def move_territories():
    df = pd.read_sql_query("SELECT * FROM Territories", nconn)
    for index, row in df.iterrows():
        try:
            query = f"""
            INSERT INTO Territories VALUES (
                '{row['TerritoryID']}',
                '{row['TerritoryDescription']}',
                {row['RegionID']}
            )
            """
            export_cursor.execute(query)
        except pyodbc.Error as e:
            print(f"Fout in Territories: {e}")
    export_conn.commit()
move_territories()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\1174576530.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM Territories", nconn)


In [35]:
def move_sales_territory():
    df = pd.read_sql_query("SELECT * FROM Sales_SalesTerritory", advconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Sales_SalesTerritory (
                TerritoryID, Name, CountryRegionCode, [Group],
                SalesYTD, CostYTD, CostLastYear, rowguid, ModifiedDate
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            export_cursor.execute(query, 
                row['TerritoryID'],
                row['Name'],
                row['CountryRegionCode'],
                row['Group'],
                row['SalesYTD'],
                row['CostYTD'],
                row['CostLastYear'],
                row['rowguid'],
                row['ModifiedDate']
            )
        except pyodbc.Error as e:
            print(f"Fout in Sales_SalesTerritory (TerritoryID: {row['TerritoryID']}): {e}")
    export_conn.commit()
    
move_sales_territory()



In [36]:
def move_person_person():
    df = pd.read_sql_query("SELECT * FROM Person_Person", advconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Person_Person (
                BusinessEntityID, PersonType, NameStyle, Title,
                FirstName, MiddleName, LastName, Suffix,
                EmailPromotion, rowguid, ModifiedDate
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            export_cursor.execute(query,
                row['BusinessEntityID'],
                row['PersonType'],
                row['NameStyle'],
                row['Title'],
                row['FirstName'],
                row['MiddleName'],
                row['LastName'],
                row['Suffix'],
                row['EmailPromotion'],
                row['rowguid'],
                row['ModifiedDate']
            )
        except pyodbc.Error as e:
            print(f"Fout in Person_Person (BusinessEntityID: {row['BusinessEntityID']}): {e}")
    export_conn.commit()
    
move_person_person()

In [37]:
def move_sales_store():
    df = pd.read_sql_query("SELECT * FROM Sales_Store", advconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Sales_Store (
                BusinessEntityID, Name, SalesPersonID, rowguid, ModifiedDate
            ) VALUES (?, ?, ?, ?, ?)
            """
            export_cursor.execute(query,
                row['BusinessEntityID'],
                row['Name'],
                row['SalesPersonID'],
                row['rowguid'],
                row['ModifiedDate']
            )
        except pyodbc.Error as e:
            print(f"Fout in Sales_Store (BusinessEntityID: {row['BusinessEntityID']}): {e}")
    export_conn.commit()
    
move_sales_store()

In [38]:

def move_person_address():
    df = pd.read_sql_query("SELECT * FROM Person_Address", advconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Person_Address (
                AddressID, AddressLine1, City, StateProvinceID,
                PostalCode, rowguid, ModifiedDate
            ) VALUES (?, ?, ?, ?, ?, ?, ?)
            """
            export_cursor.execute(query,
                row['AddressID'],
                row['AddressLine1'],
                row['City'],
                row['StateProvinceID'],
                row['PostalCode'],
                row['rowguid'],
                row['ModifiedDate']
            )
        except pyodbc.Error as e:
            print(f"Fout in Person_Address (AddressID: {row['AddressID']}): {e}")
    export_conn.commit()
    
move_person_address()

In [39]:
def move_sales_customer():
    df = pd.read_sql_query("""
        SELECT CustomerID, PersonID, StoreID, TerritoryID,
               CAST(AccountNumber AS NVARCHAR(50)) AS AccountNumber,
               rowguid, ModifiedDate
        FROM Sales_Customer
    """, advconn)

    for _, row in df.iterrows():
        try:
            export_cursor.execute("""
                INSERT INTO Sales_Customer VALUES (?, ?, ?, ?, ?, ?, ?)
            """, 
            row['CustomerID'],
            row['PersonID'] if pd.notna(row['PersonID']) else None,
            row['StoreID'] if pd.notna(row['StoreID']) else None,
            row['TerritoryID'] if pd.notna(row['TerritoryID']) else None,
            str(row['AccountNumber']) if pd.notna(row['AccountNumber']) else None,
            row['rowguid'],
            row['ModifiedDate']
            )
        except Exception as e:
            print(f"Fout bij CustomerID {row['CustomerID']}: {str(e)[:100]}...")

    export_conn.commit()
move_sales_customer()

In [40]:
def move_customer_demographics():
    df = pd.read_sql_query("SELECT * FROM CustomerDemographics", nconn)
    for _, row in df.iterrows():
        try:
            export_cursor.execute("INSERT INTO CustomerDemographics VALUES (?, ?)", 
                               row['CustomerTypeID'], 
                               row['CustomerDesc'])
        except Exception as e:
            print(f"Fout bij CustomerTypeID .")
    export_conn.commit()
    print(f"CustomerDemographics klaar: {len(df)} rijen")
move_customer_demographics()

CustomerDemographics klaar: 0 rijen


C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\3912574070.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM CustomerDemographics", nconn)


In [41]:
def move_customer():
    
    n_df = pd.read_sql_query("""
        SELECT 
            CustomerID,
            CompanyName,
            ContactName,
            ContactTitle,
            Address,
            City,
            Region,
            PostalCode,
            Country,
            Phone,
            Fax,
            NULL AS Fname,
            NULL AS Lname,
            NULL AS State
        FROM Customers
    """, nconn)
    
    aen_df = pd.read_sql_query("""
        SELECT 
            id AS CustomerID,
            company_name AS CompanyName,
            NULL AS ContactName,
            NULL AS ContactTitle,
            address AS Address,
            city AS City,
            NULL AS Region,
            zip AS PostalCode,
            NULL AS Country,
            phone AS Phone,
            NULL AS Fax,
            fname AS Fname,
            lname AS Lname,
            state AS State
        FROM Customer
    """, aenconn)
    
    combined_df = pd.concat([n_df, aen_df], ignore_index=True)

    for _, row in combined_df.iterrows():
        try:
            export_cursor.execute("""
                INSERT INTO Customer (
                    CustomerID, CompanyName, ContactName, ContactTitle,
                    Address, City, Region, PostalCode, Country,
                    Phone, Fax, Fname, Lname, State
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, 
                row['CustomerID'],
                row['CompanyName'],
                row['ContactName'] if pd.notna(row['ContactName']) else None,
                row['ContactTitle'] if pd.notna(row['ContactTitle']) else None,
                row['Address'] if pd.notna(row['Address']) else None,
                row['City'] if pd.notna(row['City']) else None,
                row['Region'] if pd.notna(row['Region']) else None,
                row['PostalCode'] if pd.notna(row['PostalCode']) else None,
                row['Country'] if pd.notna(row['Country']) else None,
                row['Phone'] if pd.notna(row['Phone']) else None,
                row['Fax'] if pd.notna(row['Fax']) else None,
                row['Fname'] if pd.notna(row['Fname']) else None,
                row['Lname'] if pd.notna(row['Lname']) else None,
                row['State'] if pd.notna(row['State']) else None
            )
        except Exception as e:
            print(f"Fout bij CustomerID {row['CustomerID']}: {str(e)[:100]}...")
    
    export_conn.commit()
    print(f"Customer migratie voltooid. {len(combined_df)} rijen toegevoegd.")

move_customer()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\2187607761.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  n_df = pd.read_sql_query("""


Customer migratie voltooid. 217 rijen toegevoegd.


In [42]:
def move_customer_customer_demo():
    df = pd.read_sql_query("SELECT * FROM CustomerCustomerDemo", nconn)
    for _, row in df.iterrows():
        try:
            export_cursor.execute("INSERT INTO CustomerCustomerDemo VALUES (?, ?)", 
                               row['CustomerID'], 
                               row['CustomerTypeID'])
        except Exception as e:
            print(f"Fout bij combinatie {row['CustomerID']}-{row['CustomerTypeID']}: {str(e)[:100]}...")
    export_conn.commit()

move_customer_customer_demo()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\4134974665.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM CustomerCustomerDemo", nconn)


In [43]:
def move_sales_order_header():
    df = pd.read_sql_query("""
        SELECT 
            SalesOrderID, RevisionNumber, OrderDate, DueDate, ShipDate,
            Status, OnlineOrderFlag, SalesOrderNumber, PurchaseOrderNumber,
            AccountNumber, CustomerID, SalesPersonID, TerritoryID,
            BillToAddressID, ShipToAddressID, ShipMethodID, CreditCardID,
            CreditCardApprovalCode, CurrencyRateID, SubTotal, TaxAmt,
            Freight, TotalDue, Comment, rowguid, ModifiedDate
        FROM Sales_SalesOrderHeader
    """, advconn)  
    success_count = 0
    for _, row in df.iterrows():
        try:
            query = """
            INSERT INTO Sales_SalesOrderHeader (
                SalesOrderID, RevisionNumber, OrderDate, DueDate, ShipDate,
                Status, OnlineOrderFlag, SalesOrderNumber, PurchaseOrderNumber,
                AccountNumber, CustomerID, SalesPersonID, TerritoryID,
                BillToAddressID, ShipToAddressID, ShipMethodID, CreditCardID,
                CreditCardApprovalCode, CurrencyRateID, SubTotal, TaxAmt,
                Freight, TotalDue, Comment, rowguid, ModifiedDate
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            
            params = (
                row['SalesOrderID'],
                row['RevisionNumber'],
                row['OrderDate'] if pd.notna(row['OrderDate']) else None,
                row['DueDate'] if pd.notna(row['DueDate']) else None,
                row['ShipDate'] if pd.notna(row['ShipDate']) else None,
                row['Status'],
                row['OnlineOrderFlag'],
                row['SalesOrderNumber'] if pd.notna(row['SalesOrderNumber']) else None,
                row['PurchaseOrderNumber'] if pd.notna(row['PurchaseOrderNumber']) else None,
                row['AccountNumber'] if pd.notna(row['AccountNumber']) else None,
                row['CustomerID'] if pd.notna(row['CustomerID']) else None,
                row['SalesPersonID'] if pd.notna(row['SalesPersonID']) else None,
                row['TerritoryID'] if pd.notna(row['TerritoryID']) else None,
                row['BillToAddressID'] if pd.notna(row['BillToAddressID']) else None,
                row['ShipToAddressID'] if pd.notna(row['ShipToAddressID']) else None,
                row['ShipMethodID'] if pd.notna(row['ShipMethodID']) else None,
                row['CreditCardID'] if pd.notna(row['CreditCardID']) else None,
                row['CreditCardApprovalCode'] if pd.notna(row['CreditCardApprovalCode']) else None,
                row['CurrencyRateID'] if pd.notna(row['CurrencyRateID']) else None,
                float(row['SubTotal']) if pd.notna(row['SubTotal']) else 0.0,
                float(row['TaxAmt']) if pd.notna(row['TaxAmt']) else 0.0,
                float(row['Freight']) if pd.notna(row['Freight']) else 0.0,
                float(row['TotalDue']) if pd.notna(row['TotalDue']) else 0.0,
                row['Comment'] if pd.notna(row['Comment']) else None,
                row['rowguid'],
                row['ModifiedDate'] if pd.notna(row['ModifiedDate']) else None
            )
            
            export_cursor.execute(query, params)
            success_count += 1
            
        except pyodbc.Error as e:
            print(f"Fout bij SalesOrderID {row['SalesOrderID']}: {str(e)[:200]}")
        except Exception as e:
            print(f"Onverwachte fout bij SalesOrderID {row['SalesOrderID']}: {str(e)[:200]}")
    
    export_conn.commit()
    
move_sales_order_header()

In [44]:
def move_product():
    aen_df = pd.read_sql_query("""
        SELECT 
            id AS ProductID,
            name AS Name,
            description AS Description,
            prod_size AS Prod_size,
            color AS Color,
            quantity AS Quantity,
            unit_price AS UnitPrice,
            picture_name AS Picture_name,
            Category AS Category,
            NULL AS SupplierID,
            NULL AS CategoryID,
            NULL AS QuantityPerUnit,
            NULL AS UnitsInStock,
            NULL AS UnitsOnOrder,
            NULL AS ReorderLevel,
            0 AS Discontinued
        FROM Product
    """, aenconn)

    n_df = pd.read_sql_query("""
        SELECT 
            ProductID,
            ProductName AS Name,
            NULL AS Description,
            NULL AS Prod_size,
            NULL AS Color,
            NULL AS Quantity,
            UnitPrice,
            NULL AS Picture_name,
            NULL AS Category,
            SupplierID,
            CategoryID,
            QuantityPerUnit,
            UnitsInStock,
            UnitsOnOrder,
            ReorderLevel,
            Discontinued
        FROM Products
    """, nconn)

    combined_df = pd.concat([aen_df, n_df], ignore_index=True)

    for _, row in combined_df.iterrows():
        try:
            query = """
            INSERT INTO Product (
                ProductID, Name, Description, Prod_size, Color,
                Quantity, unit_price, Picture_name, Category,
                SupplierID, CategoryID, QuantityPerUnit,
                UnitsInStock, UnitsOnOrder, ReorderLevel, Discontinued
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?)
            """
            params = (
                row['ProductID'],
                (row['Name']), 
                (row['Description']) if pd.notna(row['Description']) else None,
                (row['Prod_size']) if pd.notna(row['Prod_size']) else None,
                (row['Color']) if pd.notna(row['Color']) else None,
                (row['Quantity']) if pd.notna(row['Quantity']) else None,
                (row['UnitPrice']) if pd.notna(row['UnitPrice']) else 0.0,
                (row['Picture_name']) if pd.notna(row['Picture_name']) else None,
                (row['Category']) if pd.notna(row['Category']) else None,
                (row['SupplierID']) if pd.notna(row['SupplierID']) else None,
                (row['CategoryID']) if pd.notna(row['CategoryID']) else None,
                (row['QuantityPerUnit']) if pd.notna(row['QuantityPerUnit']) else None,
                (row['UnitsInStock']) if pd.notna(row['UnitsInStock']) else None,
                (row['UnitsOnOrder']) if pd.notna(row['UnitsOnOrder']) else None,
                (row['ReorderLevel']) if pd.notna(row['ReorderLevel']) else None,
                (row['Discontinued'])
            )
            export_cursor.execute(query, params)
        except pyodbc.Error as e:
            print(f"Fout in Product (ProductID: {row['ProductID']}): {e}")
    
    export_conn.commit()

move_product()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\1782744661.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  n_df = pd.read_sql_query("""


In [45]:
def move_production_product():
    df = pd.read_sql_query("SELECT * FROM Production_Product", advconn)
    
    date_cols = ['SellStartDate', 'SellEndDate', 'ModifiedDate']
    for col in date_cols:
        df[col] = pd.to_datetime(df[col])
    if 'rowguid' in df.columns:
        df['rowguid'] = df['rowguid'].astype(str)
    
    if 'DiscontinuedDate' in df.columns:
        df['DiscontinuedDate'] = pd.to_datetime(df['DiscontinuedDate']).dt.strftime('%Y-%m-%d')
    
    for _, row in df.iterrows():
        try:
            export_cursor.execute("""
                INSERT INTO Production_Product (
                    ProductID, Name, ProductNumber, MakeFlag, FinishedGoodsFlag,
                    Color, SafetyStockLevel, ReorderPoint, StandardCost, ListPrice,
                    Size, SizeUnitMeasureCode, WeightUnitMeasureCode, Weight,
                    DaysToManufacture, ProductLine, Class, Style, ProductSubcategoryID,
                    ProductModelID, SellStartDate, SellEndDate, DiscontinuedDate,
                    rowguid, ModifiedDate, ProductCategoryID
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, 
                row['ProductID'],
                row['Name'],
                row['ProductNumber'],
                row['MakeFlag'],
                row['FinishedGoodsFlag'],
                row['Color'] if pd.notnull(row['Color']) else None,
                row['SafetyStockLevel'],
                row['ReorderPoint'],
                row['StandardCost'],
                row['ListPrice'],
                row['Size'] if pd.notnull(row['Size']) else None,
                row['SizeUnitMeasureCode'] if pd.notnull(row['SizeUnitMeasureCode']) else None,
                row['WeightUnitMeasureCode'] if pd.notnull(row['WeightUnitMeasureCode']) else None,
                row['Weight'] if pd.notnull(row['Weight']) else None,
                row['DaysToManufacture'],
                row['ProductLine'] if pd.notnull(row['ProductLine']) else None,
                row['Class'] if pd.notnull(row['Class']) else None,
                row['Style'] if pd.notnull(row['Style']) else None,
                row['ProductSubcategoryID'] if pd.notnull(row['ProductSubcategoryID']) else None,
                row['ProductModelID'] if pd.notnull(row['ProductModelID']) else None,
                row['SellStartDate'],
                row['SellEndDate'] if pd.notnull(row['SellEndDate']) else None,
                row['DiscontinuedDate'] if pd.notnull(row['DiscontinuedDate']) else None,
                row['rowguid'],
                row['ModifiedDate'],
                row['ProductCategoryID'] if pd.notnull(row['ProductCategoryID']) else None
            )
        except Exception as e:
            print(f"Error with ProductID {row['ProductID']}: {str(e)}")
            print(f"Problematic row values: {row.to_dict()}")
    
    export_conn.commit()
    
move_production_product()

In [46]:
def move_sales_order_detail():
    df = pd.read_sql_query("SELECT * FROM Sales_SalesOrderDetail", advconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Sales_SalesOrderDetail (
                SalesOrderDetailID, SalesOrderID, CarrierTrackingNumber, OrderQty,
                ProductID, SpecialOfferID, UnitPrice, UnitPriceDiscount,
                LineTotal, ModifiedDate
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            export_cursor.execute(query,
                row['SalesOrderDetailID'],
                row['SalesOrderID'],
                row['CarrierTrackingNumber'],
                row['OrderQty'],
                row['ProductID'],
                row['SpecialOfferID'],
                row['UnitPrice'],
                row['UnitPriceDiscount'],
                row['LineTotal'],
                row['ModifiedDate']
            )
        except pyodbc.Error as e:
            print(f"Fout in Sales_SalesOrderDetail (SalesOrderDetailID: {row['SalesOrderDetailID']}): {e}")
    export_conn.commit()
move_sales_order_detail()


In [47]:
def move_bill_of_materials():
    product_ids = pd.read_sql_query("SELECT ProductID FROM Production_Product", export_conn)['ProductID'].tolist()
    
    df = pd.read_sql_query("SELECT * FROM Production_BillOfMaterials", advconn)
    
    
    df['StartDate'] = pd.to_datetime(df['StartDate']).dt.date
    df['EndDate'] = pd.to_datetime(df['EndDate'], errors='coerce').dt.date
    df['ModifiedDate'] = pd.to_datetime(df['ModifiedDate'])
    
    df['ProductAssemblyID'] = df['ProductAssemblyID'].where(
        df['ProductAssemblyID'].isin(product_ids), None)
    
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Production_BillOfMaterials (
                BillOfMaterialsID, ProductAssemblyID, ComponentID, StartDate,
                EndDate, UnitMeasureCode, BOMLevel, PerAssemblyQty, ModifiedDate
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            params = (
                int(row['BillOfMaterialsID']),
                int(row['ProductAssemblyID']) if pd.notnull(row['ProductAssemblyID']) else None,
                int(row['ComponentID']),
                row['StartDate'],
                row['EndDate'] if pd.notnull(row['EndDate']) else None,
                str(row['UnitMeasureCode']).strip(),
                int(row['BOMLevel']),
                float(row['PerAssemblyQty']),
                row['ModifiedDate']
            )
            
            export_cursor.execute(query, params)
            
        except pyodbc.Error as e:
            print(f"\nFout in rij {index} (BillOfMaterialsID: {row['BillOfMaterialsID']}): {e}")
    
    export_conn.commit()

move_bill_of_materials()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\3250266924.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_ids = pd.read_sql_query("SELECT ProductID FROM Production_Product", export_conn)['ProductID'].tolist()


In [48]:
def move_employee():
    n_df = pd.read_sql_query("""
        SELECT 
            EmployeeID, LastName, FirstName, Title, TitleOfCourtesy,
            BirthDate, HireDate, Address, City, Region, PostalCode,
            Country, HomePhone AS Phone, Extension, Photo, Notes,
            ReportsTo AS manager_id, PhotoPath,
            NULL AS dept_id, NULL AS street, NULL AS state, NULL AS zip_code,
            NULL AS status, NULL AS ss_number, NULL AS salary, NULL AS start_date,
            NULL AS Bene_health_ins, NULL AS Bene_life_ins, NULL AS Bene_day_care, NULL AS sex,
            NULL AS NationalIDNumber, NULL AS LoginID, NULL AS OrganizationNode, NULL AS OrganizationLevel,
            NULL AS JobTitle, NULL AS MaritalStatus, NULL AS Gender, NULL AS SalariedFlag,
            NULL AS VacationHours, NULL AS SickLeaveHours, NULL AS CurrentFlag, NULL AS DepartmentID, NULL AS ModifiedDate
        FROM dbo.Employees
    """, nconn)
    
    aen_df = pd.read_sql_query("""
        SELECT 
            emp_id AS EmployeeID, emp_lname AS LastName, emp_fname AS FirstName,
            NULL AS Title, NULL AS TitleOfCourtesy, birth_date AS BirthDate, start_date AS HireDate,
            street AS Address, city AS City, state AS Region, zip_code AS PostalCode,
            NULL AS Country, phone AS Phone, NULL AS Extension, NULL AS Photo, NULL AS Notes,
            manager_id, dept_id, street, state, zip_code, status, ss_number, salary, start_date,
            bene_health_ins AS Bene_health_ins, bene_life_ins AS Bene_life_ins, bene_day_care AS Bene_day_care, sex,
            NULL AS NationalIDNumber, NULL AS LoginID, NULL AS OrganizationNode, NULL AS OrganizationLevel,
            NULL AS JobTitle, NULL AS MaritalStatus, sex AS Gender, NULL AS SalariedFlag,
            NULL AS VacationHours, NULL AS SickLeaveHours, NULL AS CurrentFlag, dept_id AS DepartmentID, NULL AS ModifiedDate, NULL AS PhotoPath
        FROM Employee
    """, aenconn)
    
    adv_df = pd.read_sql_query("""
        SELECT 
            BusinessEntityID AS EmployeeID, NULL AS LastName, NULL AS FirstName,
            JobTitle AS Title, NULL AS TitleOfCourtesy, BirthDate, HireDate,
            NULL AS Address, NULL AS City, NULL AS Region, NULL AS PostalCode,
            NULL AS Country, NULL AS Phone, NULL AS Extension, NULL AS Photo, NULL AS Notes,
            NULL AS manager_id, NULL AS dept_id, NULL AS street, NULL AS state, NULL AS zip_code,
            NULL AS status, NULL AS ss_number, NULL AS salary, NULL AS start_date,
            NULL AS Bene_health_ins, NULL AS Bene_life_ins, NULL AS Bene_day_care, NULL AS sex,
            NationalIDNumber, LoginID, OrganizationNode, OrganizationLevel,
            JobTitle, MaritalStatus, Gender, SalariedFlag,
            VacationHours, SickLeaveHours, CurrentFlag, DepartmentID, ModifiedDate, NULL AS PhotoPath
        FROM HumanResources_Employee
    """, advconn)
    
    combined_df = pd.concat([n_df, aen_df, adv_df], ignore_index=True)
    final_df = combined_df.groupby('EmployeeID', as_index=False).first()
    
    all_employee_ids = set(final_df['EmployeeID'])
   
    try:
        export_cursor.execute("ALTER TABLE Employee NOCHECK CONSTRAINT FK__Employee__manage__5BE2A6F2")
        export_conn.commit()
        print("Foreign key constraint tijdelijk uitgeschakeld")
    except Exception as e:
        print(f"Kon foreign key constraint niet uitschakelen: {str(e)}")

    for _, row in final_df.iterrows():
        try:
            start_date = str(row['start_date']) if pd.notna(row['start_date']) else None
            
            export_cursor.execute("""
                INSERT INTO Employee (
                    EmployeeID, LastName, FirstName, Title, TitleOfCourtesy,
                    BirthDate, HireDate, Address, City, Region, PostalCode,
                    Country, Phone, Extension, Photo, Notes, PhotoPath,
                    manager_id, dept_id, street, state, [zip-code], status,
                    [ss-number], salary, start_date, Bene_health_ins, Bene_life_ins,
                    Bene_day_care, sex, NationalIDNumber, LoginID, OrganizationNode,
                    OrganizationLevel, JobTitle, MaritalStatus, Gender, SalariedFlag,
                    VacationHours, SickLeaveHours, CurrentFlag, DepartmentID, ModifiedDate
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
                          ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, 
                row['EmployeeID'],
                row['LastName'] if pd.notna(row['LastName']) else None,
                row['FirstName'] if pd.notna(row['FirstName']) else None,
                row['Title'] if pd.notna(row['Title']) else None,
                row['TitleOfCourtesy'] if pd.notna(row['TitleOfCourtesy']) else None,
                row['BirthDate'] if pd.notna(row['BirthDate']) else None,
                row['HireDate'] if pd.notna(row['HireDate']) else None,
                row['Address'] if pd.notna(row['Address']) else None,
                row['City'] if pd.notna(row['City']) else None,
                row['Region'] if pd.notna(row['Region']) else None,
                row['PostalCode'] if pd.notna(row['PostalCode']) else None,
                row['Country'] if pd.notna(row['Country']) else None,
                row['Phone'] if pd.notna(row['Phone']) else None,
                row['Extension'] if pd.notna(row['Extension']) else None,
                row['Photo'] if pd.notna(row['Photo']) else None,
                row['Notes'] if pd.notna(row['Notes']) else None,
                row['PhotoPath'] if pd.notna(row['PhotoPath']) else None,
                None,  # Manager_id eerst op NULL zetten
                row['dept_id'] if pd.notna(row['dept_id']) else None,
                str(row['street']) if pd.notna(row['street']) else None,
                str(row['state']) if pd.notna(row['state']) else None,
                str(row['zip_code']) if pd.notna(row['zip_code']) else None,
                str(row['status']) if pd.notna(row['status']) else None,
                row['ss_number'] if pd.notna(row['ss_number']) else None,
                float(row['salary']) if pd.notna(row['salary']) else None,
                start_date,
                str(row['Bene_health_ins']) if pd.notna(row['Bene_health_ins']) else None,
                str(row['Bene_life_ins']) if pd.notna(row['Bene_life_ins']) else None,
                str(row['Bene_day_care']) if pd.notna(row['Bene_day_care']) else None,
                str(row['sex']) if pd.notna(row['sex']) else None,
                str(row['NationalIDNumber']) if pd.notna(row['NationalIDNumber']) else None,
                str(row['LoginID']) if pd.notna(row['LoginID']) else None,
                str(row['OrganizationNode']) if pd.notna(row['OrganizationNode']) else None,
                row['OrganizationLevel'] if pd.notna(row['OrganizationLevel']) else None,
                str(row['JobTitle']) if pd.notna(row['JobTitle']) else None,
                str(row['MaritalStatus']) if pd.notna(row['MaritalStatus']) else None,
                str(row['Gender']) if pd.notna(row['Gender']) else None,
                row['SalariedFlag'] if pd.notna(row['SalariedFlag']) else None,
                row['VacationHours'] if pd.notna(row['VacationHours']) else None,
                row['SickLeaveHours'] if pd.notna(row['SickLeaveHours']) else None,
                row['CurrentFlag'] if pd.notna(row['CurrentFlag']) else None,
                row['DepartmentID'] if pd.notna(row['DepartmentID']) else None,
                row['ModifiedDate'] if pd.notna(row['ModifiedDate']) else None
            )
        except Exception as e:
            print(f"Fout bij EmployeeID {row['EmployeeID']}: {str(e)}")
            print(f"Problematische rij data: {row.to_dict()}")
    
    export_conn.commit()
    print(f"Eerste insert ronde voltooid. {len(final_df)} rijen toegevoegd.")
    
    # Update manager relaties voor alle employees waar dit van toepassing is
    for _, row in final_df.iterrows():
        if pd.notna(row['manager_id']) and row['manager_id'] in all_employee_ids:
            try:
                export_cursor.execute("""
                    UPDATE Employee 
                    SET manager_id = ?
                    WHERE EmployeeID = ?
                """, 
                    row['manager_id'],
                    row['EmployeeID']
                )
            except Exception as e:
                print(f"Fout bij update manager voor EmployeeID {row['EmployeeID']}: {str(e)}")
    
    export_conn.commit()
    print("Manager relaties bijgewerkt.")
    
    # Schakel de foreign key constraint weer in
    try:
        export_cursor.execute("ALTER TABLE Employee CHECK CONSTRAINT FK__Employee__manage__5BE2A6F2")
        export_conn.commit()
        print("Foreign key constraint weer ingeschakeld")
        
        # Valideer de constraints
        export_cursor.execute("DBCC CHECKCONSTRAINTS('Employee')")
        result = export_cursor.fetchall()
        if result:
            print("Waarschuwing: Er zijn constraint violations gevonden:")
            for row in result:
                print(row)
        else:
            print("Alle constraints zijn geldig")
    except Exception as e:
        print(f"Kon foreign key constraint niet inschakelen: {str(e)}")
    
    

move_employee()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\3272414051.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  n_df = pd.read_sql_query("""
C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\3272414051.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([n_df, aen_df, adv_df], ignore_index=True)


Foreign key constraint tijdelijk uitgeschakeld
Eerste insert ronde voltooid. 351 rijen toegevoegd.
Manager relaties bijgewerkt.
Foreign key constraint weer ingeschakeld
Kon foreign key constraint niet inschakelen: No results.  Previous SQL was not a query.


In [49]:
def move_employee_territories():
    df = pd.read_sql_query("SELECT * FROM EmployeeTerritories", nconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO EmployeeTerritories (
                EmployeeID, TerritoryID
            ) VALUES (?, ?)
            """
            export_cursor.execute(query,
                row['EmployeeID'],
                row['TerritoryID']
            )
        except pyodbc.Error as e:
            print(f"Fout in EmployeeTerritories (EmployeeID: {row['EmployeeID']}, TerritoryID: {row['TerritoryID']}): {e}")
    export_conn.commit()
move_employee_territories()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\1179525313.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM EmployeeTerritories", nconn)


In [50]:
def move_bonus():
    df = pd.read_sql_query("SELECT * FROM Bonus", aenconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Bonus (
                Emp_id, Bonus_date, Bonus_amount
            ) VALUES (?, ?, ?)
            """
            export_cursor.execute(query,
                row['emp_id'],
                row['bonus_date'],
                row['bonus_amount']
            )
        except pyodbc.Error as e:
            print(f"Fout in Bonus (Emp_id: {row['Emp_id']}): {e}")
    export_conn.commit()
move_bonus()

In [51]:
def move_orders():
    # Haal gegevens uit de eerste bron (nconn)
    n_df = pd.read_sql_query("""
        SELECT 
            OrderID, CustomerID, EmployeeID, 
            OrderDate, RequiredDate, ShippedDate, 
            ShipVia, Freight, 
            ShipName, ShipAddress, ShipCity, 
            ShipRegion, ShipPostalCode, ShipCountry,
            NULL AS Region, NULL AS Sales_rep
        FROM dbo.Orders
    """, nconn)
    
    # Haal gegevens uit de tweede bron (aenconn) met correcte mapping
    aen_df = pd.read_sql_query("""
        SELECT 
            id AS OrderID, 
            cust_id AS CustomerID,  -- cust_id wordt gemapped naar CustomerID
            sales_rep AS EmployeeID,  -- sales_rep wordt EmployeeID
            order_date AS OrderDate,  -- order_date wordt OrderDate
            NULL AS RequiredDate, 
            NULL AS ShippedDate,
            NULL AS ShipVia, 
            NULL AS Freight,
            NULL AS ShipName, 
            NULL AS ShipAddress, 
            NULL AS ShipCity,
            region AS ShipRegion, 
            NULL AS ShipPostalCode, 
            NULL AS ShipCountry,
            region AS Region, 
            sales_rep AS Sales_rep
        FROM Sales_Order
    """, aenconn)
    
    # Combineer beide dataframes
    combined_df = pd.concat([n_df, aen_df], ignore_index=True)
    
    # Verwijder mogelijke duplicates op OrderID
    final_df = combined_df.drop_duplicates(subset=['OrderID'], keep='first')
    
    # Insert logica
    for _, row in final_df.iterrows():
        try:
            export_cursor.execute("""
                INSERT INTO [Order] (
                    OrderID, CustomerID, EmployeeID, 
                    OrderDate, RequiredDate, ShippedDate,
                    ShipVia, Freight, ShipName, ShipAdress,
                    ShipCity, ShipPostalCode, ShipCountry,
                    Region, Sales_rep
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, 
                row['OrderID'],
                row['CustomerID'] if pd.notna(row['CustomerID']) else None,
                row['EmployeeID'] if pd.notna(row['EmployeeID']) else None,
                row['OrderDate'] if pd.notna(row['OrderDate']) else None,
                row['RequiredDate'] if pd.notna(row['RequiredDate']) else None,
                row['ShippedDate'] if pd.notna(row['ShippedDate']) else None,
                row['ShipVia'] if pd.notna(row['ShipVia']) else None,
                float(row['Freight']) if pd.notna(row['Freight']) else None,
                str(row['ShipName']) if pd.notna(row['ShipName']) else None,
                str(row['ShipAddress']) if pd.notna(row['ShipAddress']) else None,
                str(row['ShipCity']) if pd.notna(row['ShipCity']) else None,
                str(row['ShipPostalCode']) if pd.notna(row['ShipPostalCode']) else None,
                str(row['ShipCountry']) if pd.notna(row['ShipCountry']) else None,
                str(row['Region']) if pd.notna(row['Region']) else None,
                row['Sales_rep'] if pd.notna(row['Sales_rep']) else None
            )
        except pyodbc.Error as e:
            print(f"Fout bij OrderID {row['OrderID']}: {str(e)}")
            print(f"Problematische rij data: {row.to_dict()}")
    
    export_conn.commit()
    
move_orders()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\543909347.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  n_df = pd.read_sql_query("""
C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\543909347.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([n_df, aen_df], ignore_index=True)


In [52]:
def move_order_details():
    df = pd.read_sql_query("SELECT * FROM [Order Details]", nconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Order_Details (
                OrderID, ProductID, UnitPrice, Quantity, Discount
            ) VALUES (?, ?, ?, ?, ?)
            """
            export_cursor.execute(query,
                row['OrderID'],
                row['ProductID'],
                row['UnitPrice'],
                row['Quantity'],
                row['Discount']
            )
        except pyodbc.Error as e:
            print(f"Fout in Order_Details (OrderID: {row['OrderID']}, ProductID: {row['ProductID']}): {e}")
    export_conn.commit()

move_order_details()

C:\Users\ibrah\AppData\Local\Temp\ipykernel_9068\2567412267.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM [Order Details]", nconn)


In [53]:
def move_sales_order_item():
    df = pd.read_sql_query("SELECT * FROM Sales_Order_Item", aenconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Sales_Order_Item (
                Id, Line_id, Prod_id, Quantity, Ship_date
            ) VALUES (?, ?, ?, ?, ?)
            """
            export_cursor.execute(query,
                row['id'],
                row['line_id'],
                row['prod_id'],
                row['quantity'],
                row['ship_date']
            )
        except pyodbc.Error as e:
            print(f"Fout in Sales_Order_Item (Id: {row['Id']}, Line_id: {row['Line_id']}): {e}")
    export_conn.commit()
move_sales_order_item()

In [54]:
def move_purchasing_vendor():
    df = pd.read_sql_query("SELECT * FROM Purchasing_Vendor", advconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Purchasing_Vendor (
                BusinessEntityID, AccountNumber, Name, CreditRating,
                PreferredVendorStatus, ActiveFlag, PurchasingWebServiceURL, ModifiedDate
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """
            export_cursor.execute(query,
                row['BusinessEntityID'],
                row['AccountNumber'],
                row['Name'],
                row['CreditRating'],
                row['PreferredVendorStatus'],
                row['ActiveFlag'],
                row['PurchasingWebServiceURL'],
                row['ModifiedDate']
            )
        except pyodbc.Error as e:
            print(f"Fout in Purchasing_Vendor (BusinessEntityID: {row['BusinessEntityID']}): {e}")
    export_conn.commit()

def move_purchasing_purchase_order_header():
    df = pd.read_sql_query("SELECT * FROM Purchasing_PurchaseOrderHeader", advconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Purchasing_PurchaseOrderHeader (
                PurchaseOrderID, RevisionNumber, Status, EmployeeID,
                VendorID, ShipMethodID, OrderDate, ShipDate,
                SubTotal, TaxAmt, Freight, TotalDue, ModifiedDate
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            export_cursor.execute(query,
                row['PurchaseOrderID'],
                row['RevisionNumber'],
                row['Status'],
                row['EmployeeID'],
                row['VendorID'],
                row['ShipMethodID'],
                row['OrderDate'],
                row['ShipDate'],
                row['SubTotal'],
                row['TaxAmt'],
                row['Freight'],
                row['TotalDue'],
                row['ModifiedDate']
            )
        except pyodbc.Error as e:
            print(f"Fout in Purchasing_PurchaseOrderHeader (PurchaseOrderID: {row['PurchaseOrderID']}): {e}")
    export_conn.commit()

def move_purchasing_purchase_order_detail():
    df = pd.read_sql_query("SELECT * FROM Purchasing_PurchaseOrderDetail", advconn)
    for index, row in df.iterrows():
        try:
            query = """
            INSERT INTO Purchasing_PurchaseOrderDetail (
                PurchaseOrderDetailID, PurchaseOrderID, DueDate, OrderQty,
                ProductID, UnitPrice, LineTotal, ReceivedQty,
                RejectedQty, StockedQty, ModifiedDate
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            export_cursor.execute(query,
                row['PurchaseOrderDetailID'],
                row['PurchaseOrderID'],
                row['DueDate'],
                row['OrderQty'],
                row['ProductID'],
                row['UnitPrice'],
                row['LineTotal'],
                row['ReceivedQty'],
                row['RejectedQty'],
                row['StockedQty'],
                row['ModifiedDate']
            )
        except pyodbc.Error as e:
            print(f"Fout in Purchasing_PurchaseOrderDetail (PurchaseOrderDetailID: {row['PurchaseOrderDetailID']}): {e}")
    export_conn.commit()

move_purchasing_vendor()
move_purchasing_purchase_order_header()
move_purchasing_purchase_order_detail()


In [55]:
export_cursor.close()
export_conn.close()
nconn_cursor.close()
nconn.close()
aenconn.close()